In [21]:
%load_ext autoreload
%autoreload 2

from sklearn.covariance import ledoit_wolf
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.utils.extmath import fast_logdet, randomized_svd, svd_flip
from torchvision import models, transforms

from meanshift import *


def receptive_field_for_layer(net, L=4, shape=(3, 224, 224)):
    
    net = net[:L]
    I = shape[1]
    F, H = net(torch.ones((1, *shape))).shape[1:3] # features, height
    
    for i in np.concatenate([np.arange(0, shape[1]+1, 2)[:0:-1], [1]]):
        x = torch.ones((2, 3, i, i))
        o = net(x)
        #print(i, o.shape)        
        if o.shape[2] == 1:
            break
    
    P = i # patch size
    
    if H > 1:
        s = int((I - P) / (H - 1) // 2 * 2) # stride
    else:
        s = -1
    
    return P, s, H, F


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
net, layer_map = get_fmap("eff")

ds = []
for l in layer_map:
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,1,2,2,112,48
1,2,2,2,112,24
2,3,4,4,56,32
3,4,8,8,28,56
4,5,16,16,14,112
5,6,16,16,14,160
6,7,32,32,7,272


In [7]:
net, layer_map = get_fmap("vgg")

ds = []
for l in layer_map:
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,5,2,2,112,64
1,10,6,2,56,128
2,19,14,6,28,256
3,27,14,6,28,512
4,33,30,14,14,512
5,34,30,14,14,512


In [12]:
net, layer_map = get_fmap("vgg2")

ds = []
for l in layer_map:
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,7,2,2,112,64
1,14,6,2,56,128
2,27,14,6,28,256
3,39,14,6,28,512
4,47,30,14,14,512
5,52,30,14,14,512


In [23]:
net, layer_map = get_fmap("res")

ds = []
for l in layer_map:
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,3,2,2,112,64
1,4,4,4,56,64
2,5,4,4,56,256
3,6,8,8,28,512
4,7,16,16,14,1024
5,8,32,32,7,2048
6,9,224,-1,1,2048


In [6]:
net = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)
net = nn.Sequential(net.conv1, net.bn1, net.relu, net.maxpool, net.layer1, net.layer2, net.layer3, net.layer4)

ds = []
for l in [1,5,6,7,8]:
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

Using cache found in /home/matthias/.cache/torch/hub/pytorch_vision_v0.9.0


,l,P,s,H,F
0,1,2,2,112,64
1,5,4,4,56,256
2,6,8,8,28,512
3,7,16,16,14,1024
4,8,32,32,7,2048


In [135]:
net = models.efficientnet_b4(pretrained=True).features
net = net.eval()

ds = []
for l in [1,2,3,4,5,6,7]:
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,1,2,2,112,48
1,2,2,2,112,24
2,3,4,4,56,32
3,4,8,8,28,56
4,5,16,16,14,112
5,6,16,16,14,160
6,7,32,32,7,272


In [15]:
net = VGG('VGG19', True).features

ds = []
for l in range(1, len(net)):
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,1,1,0,224,64
1,2,1,0,224,64
2,3,1,0,224,64
3,4,1,0,224,64
4,5,1,0,224,64
5,6,1,0,224,64
6,7,2,2,112,64
7,8,2,2,112,128
8,9,2,2,112,128
9,10,2,2,112,128


In [151]:
net = models.vgg19(pretrained=True).features

ds = []
for l in range(1, len(net)):
    ds.append((l, *receptive_field_for_layer(net, l, (3, 224, 224))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,1,1,0,224,64
1,2,1,0,224,64
2,3,1,0,224,64
3,4,1,0,224,64
4,5,2,2,112,64
5,6,2,2,112,128
6,7,2,2,112,128
7,8,2,2,112,128
8,9,2,2,112,128
9,10,6,2,56,128


In [156]:
net = VGG('VGG19', True).features

ds = []
for l in range(1, 52):
    ds.append((l, *receptive_field_for_layer(net, l, (3, 32, 32))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,1,1,0,32,64
1,2,1,0,32,64
2,3,1,0,32,64
3,4,1,0,32,64
4,5,1,0,32,64
5,6,1,0,32,64
6,7,2,2,16,64
7,8,2,2,16,128
8,9,2,2,16,128
9,10,2,2,16,128


In [158]:
net = models.vgg19(pretrained=True).features

ds = []
for l in range(1, len(net)):
    ds.append((l, *receptive_field_for_layer(net, l, (3, 32, 32))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,1,1,0,32,64
1,2,1,0,32,64
2,3,1,0,32,64
3,4,1,0,32,64
4,5,2,2,16,64
5,6,2,2,16,128
6,7,2,2,16,128
7,8,2,2,16,128
8,9,2,2,16,128
9,10,6,2,8,128


In [3]:
net = models.efficientnet_b4(pretrained=True).features

ds = []
for l in [1,2,3,4,5,6]:
    ds.append((l, *receptive_field_for_layer(net, l, (3, 32, 32))))
pd.DataFrame(ds, columns=["l", "P", "s", "H", "F"])

,l,P,s,H,F
0,1,2,2,16,48
1,2,2,2,16,24
2,3,4,4,8,32
3,4,8,8,4,56
4,5,16,16,2,112
5,6,16,16,2,160
